# Macro

In [108]:
# Widgets in this script shall be exec under Python3.7
# Unexpected error happens for 3.10

%matplotlib widget
# %matplotlib notebook

import ipywidgets
from IPython.display import display, clear_output

from scipy.integrate import odeint

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['text.usetex'] = True

# Motion equation in space colony frame

In [109]:
def motion_colony_frame(delta, t, omega):
    # Effective motion equation in Space Colony frame
    r, theta, Dr, Dtheta = delta
    # Dr, Dtheta, DDr, DDtheta, respectively
    Ddelta = np.array([
        Dr, 
        Dtheta, 
        2*omega**2*r+2*omega*Dtheta, 
        -omega**2*theta-2*omega*Dr
        ])
    return Ddelta


def sim_Track_Colony(
    v      = 1.0,
    Tmax    = 10.0,
    omega   = 1.0,
    c       = 1.0,
    plot_display = True,
):
    # Simulator of motion in Space Colony frame, returns the fig
    tSeries_Colony = np.linspace(
        # Time series
        0,
        Tmax/omega,
        1000
    )
    init_Colony = (
        # r0, theta0, Dr0, Dtheta0, respectively
        c,
        0.0,
        0.0,
        -c*omega*v
    )

    # Track simulated
    track_Colony = odeint(
        motion_colony_frame,
        init_Colony,
        tSeries_Colony,
        args=(
            omega,
        )
    )
    X, Y = track_Colony[:, 0], track_Colony[:, 1]

    # Plot
    fig_Colony, ax_Colony = plt.subplots(
        1, 1
    )
    ax_Colony.plot(X, Y, label='Colony')
    ax_Colony.legend()

    # if plot_display==False:
    #     fig_Colony.clf()

    return fig_Colony

# Interactive

In [110]:
# Layouts / Styles
layout_row = ipywidgets.Layout(
    display         = 'flex',
    flex_flow       = 'row',
    align_items     = 'center',
    justify_content = 'center',
)
layout_column = ipywidgets.Layout(
    display         = 'flex',
    flex_flow       = 'column',
    align_items     = 'center',
    justify_content = 'center',
)


# Output and display widgets
output_widget = ipywidgets.Output()
box_plot = ipywidgets.VBox(
    children = [output_widget],
    layout = layout_column
)
# display(box_plot)


# information widgets
prompt_Default = 'Ready to simulate'
box_prompt = ipywidgets.VBox(
    children = [ipywidgets.Label(prompt_Default)],
    layout = layout_column
)


# Control widgets, toggle button
toggle_control = ipywidgets.ToggleButtons(
    options = [
        'Paras',
        'Reset',
        'Hide',
        'Sim',
        'Export',
    ],
    tooltips = [
        'Set paras',
        'Reset default paras',
        'Hide paras control',
        'Run simulation',
        'Export to file',
    ],
    value = 'Reset',
    description = 'Control',
    disabled = False,
    button_style = 'primary', # 'success', 'info', 'warning', 'danger' or ''
#     icons=['check'] * 2
)
box_control = ipywidgets.HBox(
    children=[toggle_control],
    layout = layout_row
)
# display(box_control)


# Para widgets, in sliders
slider_v = ipywidgets.FloatSlider(
    value   = 2.75,
    min     = 0.0,
    max     = 10.0,
    step    = 0.01,
    description = 'V',
)
slider_Tmax = ipywidgets.FloatSlider(
    value   = 5.0,
    min     = 0.0,
    max     = 10.0,
    step    = 0.01,
    description = 'Time simed',
)
slider_omega = ipywidgets.FloatSlider(
    value   = 1.0,
    min     = 0.0,
    max     = 10.0,
    step    = 0.01,
    description = 'omega',
)
slider_c = ipywidgets.FloatSlider(
    value   = 1.0,
    min     = 0.0,
    max     = 10.0,
    step    = 0.01,
    description = 'c',
)
sliders_paras = [
    slider_v,
    slider_Tmax,
    slider_omega,
    slider_c,
]
box_paras = ipywidgets.VBox(
    children = sliders_paras,
    layout = layout_column
)


# Assemble together
panel_Track_Colony = ipywidgets.VBox(
    [
        box_control,
        box_paras,
        box_plot,
        box_prompt,
    ]
)

savefig_prefix = 'Plot_TrackColony_'
def handle_toggle_savefig(change):
    if change['new']=='.pdf':
        savefig_suffix = '.pdf'
    elif change['new']=='.jpg':
        savefig_suffix = '.jpg'
toggle_savefig = ipywidgets.ToggleButtons(
    options = [
        '.pdf',
        '.jpg',
    ],
    value = None,
    # layout=widgets.Layout(width=widget_width, padding=padding),
    # style={"description_width": "initial"},
)
box_savefig = ipywidgets.HBox(
    children = [toggle_savefig],
    layout = layout_column
)

def handle_toggle_control(change):
    # With 'Set', show control and para panels
    if change['new']=='Paras':
        box_prompt = ipywidgets.Label('Setup Paras')
        panel_Track_Colony.children=[
            box_control,
            box_paras,
            box_prompt,
        ]
    # with 'Default, reset para panels
    elif change['new']=='Reset':
        slider_v.value     = 2.75
        slider_Tmax.value   = 5.0
        slider_omega.value  = 1.0
        slider_c.value      = 1.0
        box_prompt = ipywidgets.Label('Paras reset to default')
        panel_Track_Colony.children=[
            box_control,
            box_paras,
        ]
    # with 'Hide', hide para panels
    elif change['new']=='Hide':
        box_prompt = ipywidgets.Label('Paras panel hidden')
        panel_Track_Colony.children=[
            box_control,
            box_plot
        ]
    # with 'Sim, run simulation and display new fig
    elif change['new']=='Sim':
        with output_widget:
            output_widget.clear_output(wait=True)
            fig_track_colony = sim_Track_Colony(
                slider_v.value,
                slider_Tmax.value,
                slider_omega.value,
                slider_c.value,
                True,
            )
            display(fig_track_colony)
            plt.close()
        box_prompt = ipywidgets.Label('Simulation complete')
        panel_Track_Colony.children=[
            box_control,
            box_paras,
            box_plot,
            box_prompt
        ]
    # with 'Export', export to file
    elif change['new']=='Export':
        with output_widget:
            output_widget.clear_output(wait=True)
            fig_track_colony = sim_Track_Colony(
                slider_v.value,
                slider_Tmax.value,
                slider_omega.value,
                slider_c.value,
                True,
            )
            display(fig_track_colony)
            plt.close()
        box_prompt = ipywidgets.Label('Ready to export')
        if toggle_savefig.value:
            savefig_suffix = toggle_savefig.value
            savefig_paras = 'V' + str(slider_v.value)    \
                + 'Tmax' + str(slider_Tmax.value)   \
                    + 'Omega' + str(slider_omega.value)   \
                        + 'C' + str(slider_c.value)
            savefig_filename = savefig_prefix + savefig_paras + savefig_suffix
            fig_track_colony.savefig(savefig_filename)
            box_prompt = ipywidgets.Label(savefig_suffix)
        panel_Track_Colony.children=[
            box_control,
            box_paras,
            box_savefig,
            box_plot,
            box_prompt,
        ]

toggle_control.observe(handle_toggle_control, 'value')
toggle_savefig.observe(handle_toggle_savefig, 'value')


display(panel_Track_Colony)
